In [28]:
import pandas as pd
import random

In [2]:
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)
df = df[~df['categories'].isnull()]

In [3]:
def assign_category(x, i):
    try: 
        return x.split(',')[i]
    except:
        return ''

In [4]:
def check_category(x, words_list):
    if any(word in x for word in words_list):
        return True
    return False

In [5]:
l_cat = ['electronics', 'computers', 'Mobile Phones']
df['contoso_cat'] = df['categories'].map(lambda x: check_category(x, l_cat))

In [6]:
df = df[df.contoso_cat == True]

In [7]:
df['categories_0'] = df['categories'].map(lambda x: assign_category(x, 0))
df['categories_1'] = df['categories'].map(lambda x: assign_category(x, 1))
df['categories_2'] = df['categories'].map(lambda x: assign_category(x, 2))

In [8]:
df = df[df['categories_0'].isin(l_cat)
       | df['categories_1'].isin(l_cat)
       | df['categories_2'].isin(l_cat)]

In [9]:
df['business_id'].to_csv('businesses.csv', index=False)

In [48]:
df_reviews = pd.read_json('../../../../../../Downloads/yelp_dataset/yelp_academic_dataset_review.json', lines=True, nrows=100)

## Join business and reviews

In [14]:
df_reviews = pd.read_json('yelp_reviews_contoso_stores.json', lines=True)

In [19]:
df_stores = pd.read_csv('Contoso/Stores.csv')
df_geo = pd.read_csv('Contoso/Geography.csv')
df_stores = df_stores.merge(df_geo)
# tiendas de USA
df_stores = df_stores[df_stores['RegionCountryName'] ==  'United States']
# solo tiendas activas
df_stores = df_stores[df_stores['Status'] == 'On']

In [46]:
# nos quedamos con el numero de stores con review
n = df_reviews['business_id'].nunique()
# seleccionamos aleatoriamente n stores para asignarles reviews
random.seed(2020)
stores_index = random.sample(range(0, df_stores.shape[0]), n)
# creamos diccionario para añadir column StoreKey a las reviews
stores_selected =  df_stores['StoreKey'].iloc[stores_index]
l_map_store_reviews = dict(zip(df_reviews['business_id'].unique(), stores_selected))

In [47]:
df_reviews['StoreKey'] = df_reviews['business_id'].map(lambda x: l_map_store_reviews[x])

In [50]:
df_reviews.to_csv('yelp_reviews_contoso_stores_with_bussiness_id.csv', index=False)